<a href="https://colab.research.google.com/github/jmhuer/shift_invariant_dictionary_learning/blob/main/pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pretty_midi
!git clone https://github.com/music-x-lab/POP909-Dataset
%cd /content/POP909-Dataset/data_process
!pip install libfmp

     |████████████████████████████████| 5.6 MB 10.6 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591953 sha256=468713975cd5f30942946fb962f0fbd2a2e483a5215ea9678c400d09c0a55173
  Stored in directory: /root/.cache/pip/wheels/ad/74/7c/a06473ca8dcb63efb98c1e67667ce39d52100f837835ea18fa
Successfully built pretty-midi
Cloning into 'POP909-Dataset'...
remote: Enumerating objects: 9265, done.
remote: Counting objects: 100% (9265/9265), done.
remote: Compressing objects: 100% (8157/8157), done.
remote: Total 9265 (delta 13), reused 9245 (delta 6), pack-reused 0
Receiving objects: 100% (9265/9265), 45.75 MiB | 8.14 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/POP909-Dataset/data_process
     |████████████████████████████████| 109 kB 13.3 MB/s 
     |████████████████████████████████| 18.5 MB 305 kB/s 
     |████████████████████████████████| 787 kB 71.3 MB/s 
     |███████████████████

In [3]:
#@title Pytorch for DL

import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import torch
from torch.nn.utils import weight_norm
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)


def get_model_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

Using device:  cuda


In [4]:

import pickle
import os
import sys
# import utils
from processor import MidiEventProcessor
import pretty_midi 
import numpy as np


total = 0
def preprocess_midi(path, parts="all"):
    get_index = {
        "all"   : [0,3],
        "melody": [0,1],  
        "bridge": [1,2],  
        "chords": [2,3],  
    }
    index = get_index[parts]
    global total
    data = pretty_midi.PrettyMIDI(path)
    main_notes = []
    acc_notes = []
    for ins in data.instruments[index[0]:index[1]]:
        acc_notes.extend(ins.notes)
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)

    piano_program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    piano.notes.extend(acc_notes)
    # score = prettyn
    # mpr = MidiEventProcessor()
    # repr_seq = mpr.encode([main_notes, acc_notes])
    total += 1
    return piano.get_piano_roll()

def preprocess_pop909(midi_paths, save_dir,  parts="all"):
    save_py = []
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        if path[len(path)-5:len(path)] != ".xlsx":
          # print(' ', end='[{}]'.format(path), flush=True)
          filename = path + "/"+ path[10:14] + ".mid"
          try:
              data = torch.tensor(preprocess_midi(filename, parts=parts))
              print(data.shape)
          except KeyboardInterrupt:
              print(' Abort')
              return
          except EOFError:
              print('EOF Error')
              return
          save_py.append(data)
    return save_py
     
    
# replace the folder with your POP909 data folder
midi_paths = ["../POP909/" + d for d in os.listdir("../POP909/")] #not index

chord_train_dataset = preprocess_pop909(midi_paths=midi_paths, save_dir="midi_data/", parts="all")

melody_train_dataset = preprocess_pop909(midi_paths=midi_paths, save_dir="midi_data/", parts="melody")


torch.Size([128, 28827])
torch.Size([128, 36777])
torch.Size([128, 18184])
torch.Size([128, 17504])
torch.Size([128, 26261])
torch.Size([128, 30410])
torch.Size([128, 7598])
torch.Size([128, 15347])
torch.Size([128, 25360])
torch.Size([128, 20027])
torch.Size([128, 24894])
torch.Size([128, 21469])
torch.Size([128, 23946])
torch.Size([128, 21960])
torch.Size([128, 27992])
torch.Size([128, 27964])
torch.Size([128, 27086])
torch.Size([128, 20691])
torch.Size([128, 25147])
torch.Size([128, 30307])
torch.Size([128, 17559])
torch.Size([128, 30304])
torch.Size([128, 32633])
torch.Size([128, 22488])
torch.Size([128, 20736])
torch.Size([128, 27989])
torch.Size([128, 23793])
torch.Size([128, 24338])
torch.Size([128, 22502])
torch.Size([128, 20893])
torch.Size([128, 24736])
torch.Size([128, 25952])
torch.Size([128, 25522])
torch.Size([128, 20003])
torch.Size([128, 31649])
torch.Size([128, 27501])
torch.Size([128, 26649])
torch.Size([128, 29575])
torch.Size([128, 20549])
torch.Size([128, 17527])
t

In [5]:


def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm




In [6]:
import IPython.display

# cello_c_chord.write('cello-C-chord.mid')
# pm = piano_roll_to_pretty_midi(train_dataset[0])
# IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)



#were in bussiness

In [13]:
#@title KWTA


class SparsifyBase(nn.Module):
    def __init__(self, sparse_ratio=0.5):
        super(SparsifyBase, self).__init__()
        self.sr = sparse_ratio
        self.preact = None
        self.act = None
    def get_activation(self):
        def hook(model, input, output):
            self.preact = input[0].cpu().detach().clone()
            self.act = output.cpu().detach().clone()
        return hook
    def record_activation(self):
        self.register_forward_hook(self.get_activation())


class Sparsify1D_kactive(SparsifyBase):
    def __init__(self, k=1):
        super(Sparsify1D_kactive, self).__init__()
        self.k = k
    def forward(self, x):
        m = torch.zeros(x.shape).to(device)
        for i in range(self.k):
            # print("shape", x.shape)
            indeces = x.topk(self.k, dim=1)[1][:, i]
            m += torch.mul(torch.zeros(x.shape).to(device).scatter(1, indeces.unsqueeze(1), 1), x)
            # print("\n hi", m )
        return m.double()


class Sparsify2D_vol(SparsifyBase):
    '''cross channel sparsify'''
    def __init__(self, sparse_ratio=0.1):
        super(Sparsify2D_vol, self).__init__()
        self.sr = sparse_ratio
    def forward(self, x):
        # print("x size", x.shape)
        size = x.shape[1]*x.shape[2]*x.shape[3]
        k = int(self.sr*size)
        tmpx = x.view(x.shape[0], -1)
        # print("size after view",tmpx.shape )
        topval = tmpx.topk(k, dim=1)[0][:,-1]
        topval = topval.repeat(tmpx.shape[1], 1).permute(1,0).view_as(x)
        comp = (x>=topval).to(x)
        return comp*x

In [9]:
#@title TCN 

class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 1)
        self.conv2.weight.data.normal_(0, 1)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 1)

    def forward(self, x):
        # print("block ", x.size())
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)


        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:].size())
        # print("last layer conv", self.network[-1].conv2.weight.data[:,0,:])

    def forward(self, x):
        return self.network(x)


In [47]:
#@title TCN - Autoeconder 

class TCNAutoencoder(nn.Module):
    def __init__(self, kernel_size, sparse_ratio=0.1):
        super(TCNAutoencoder, self).__init__()
        self.wta = Sparsify2D_vol(sparse_ratio=sparse_ratio)
        # self.feature = TemporalConvNet(128, [128], kernel_size=64, dropout=dropout).double()
        self.encoder = torch.nn.Conv2d(in_channels=1, out_channels=88, kernel_size=(128,256), padding=0, bias=False, stride=12)
        self.decoder = torch.nn.ConvTranspose2d(in_channels=88, out_channels=1, kernel_size=(128,256), padding=0, bias=False, stride=12)
        # self.encoder2 = torch.nn.Conv2d(in_channels=1, out_channels=24, kernel_size=(128,12), padding=0, bias=False, stride=12)
        # self.decoder2 = torch.nn.ConvTranspose2d(in_channels=24, out_channels=1, kernel_size=(128,12), padding=0, bias=False, stride=12)
        # self.encoder2.weight.data.normal_(30)
        # self.encoder.weight.data.normal_(30)
        # self.decoder.weight.data.normal_(300)
        self.relu1 = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.code = None
        # torch.nn.init.xavier_uniform(self.encoder.weight)
        # torch.nn.init.xavier_uniform(self.decoder.weight)
    def get_kernels(self):
        return self.decoder.weight.data[:,0,:]
    def feature_map(self, x):
        code = self.code
        return code
    def forward(self, x):
        # x needs to have dimension (N, C, L) in order to be passed into CNN
        # output = self.feature(x)
        self.code = self.wta(self.sig(self.encoder(x)))
        output = self.decoder(self.code)
        # output = self.decoder(self.code )
        return output

# t = torch.rand((1, 1,128,1000))
# b = TCNAutoencoder(kernel_size = 128, wta_k=5)
# b(t)

# print(a.shape)
# np.savetxt("foo.csv", b.code[0,:,0,:].detach().numpy(), delimiter=",")


In [ ]:
#@title GO

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device: ", device)
kernel_size = 64
model = TCNAutoencoder(kernel_size=kernel_size, 
                       sparse_ratio = 0.05).to(device).double()
print("TCNAutoencoder trainable parameters: ", get_model_parameters(model))

# model.load_state_dict(torch.load("model.pth"))


loss_fn = torch.nn.MSELoss().to(device)
# optimizer = optim.SGD(model.parameters(), lr=.0008, weight_decay = 0.00001, momentum=0.05) ##this has weight decay just like you implemented
optimizer = optim.Adam(model.parameters(), lr=.001,  betas=(0.8, 0.999), eps=1e-08, weight_decay=0, amsgrad=True) ##this has weight decay just like you implemented
epochs = 500
history = {"loss": []}

chord_train_dataset

melody_train_dataset 
calc = []
total_len = 0
for i in range(epochs):
    # random.shuffle(train_dataset)
    #decaying WTA
    # if i % 5 == 0 and i != 0:
        # model.wta.k = max(2, model.wta.k - 1)
        # print("model.wta.k: ", model.wta.k)
    for chord, melody in zip(chord_train_dataset, melody_train_dataset):
        # calc.extend(train_data.flatten().numpy())
        #normalize 
        chord, melody = chord / 111.14747885919755, melody / 111.14747885919755
        #preprocess
        chord = chord[None, None, 0:(len(chord)//kernel_size)*kernel_size].to(device).double()
        melody = melody[None, None,0:(len(melody)//kernel_size)*kernel_size].to(device).double()
        # print("chrd size", chord.shape )
        # print("melody size", melody.shape )
        #preprocess
        optimizer.zero_grad()
        output = model(chord)

        loss = loss_fn(output[:,:,:,0:melody.shape[2]], chord[:,:,:,0:output.shape[2]])
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        history["loss"].append(float(loss)*len(chord))
        # print("Loss : {} ".format(float(loss)))
        total_len += len(chord)
    print("Epoch : {} \t Loss : {} ".format(i, round(float(np.mean(history["loss"], axis=0)/total_len),9)))
    history["loss"] = []
    total_len = 0

# print(len(calc))
# print(np.mean(calc, axis=0))
# print(np.std(calc, axis=0))


Using device:  cuda
TCNAutoencoder trainable parameters:  5767168
Epoch : 0 	 Loss : 1.0507e-05 
Epoch : 1 	 Loss : 1.1281e-05 
Epoch : 2 	 Loss : 6.082e-06 
Epoch : 3 	 Loss : 4.416e-06 
Epoch : 4 	 Loss : 3.633e-06 
Epoch : 5 	 Loss : 2.988e-06 
Epoch : 6 	 Loss : 2.631e-06 
Epoch : 7 	 Loss : 2.405e-06 
Epoch : 8 	 Loss : 2.261e-06 
Epoch : 9 	 Loss : 2.151e-06 
Epoch : 10 	 Loss : 1.993e-06 


In [5]:
#try 2d conv

index_example = 545
# index_example = 876

size = 512

print("orginal 1")

raw_input =  chord_train_dataset[index_example][None,:, 0:size]

pm = piano_roll_to_pretty_midi(raw_input.cpu().numpy()[0].astype(int))
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)

raw_input =  chord_train_dataset[index_example][None, None,:, 0:size]



orginal 1


NameError: ignored

In [2]:
# model = TCNAutoencoder(kernel_size=32, 
#                        dropout=0.1, 
#                        wta_k = 5).to(device).double()
# print("TCNAutoencoder trainable parameters: ", get_model_parameters(model))

# model.load_state_dict(torch.load("/content/model.pth"))


In [3]:


######reconstructed
print("reconstructed")


raw_input = raw_input  / 111.14747885919755
raw_input = raw_input.to(device).double()
# print("train_data size", train_datase.shape)
model_out = model(raw_input)
model_out = model_out * 111.14747885919755
# model_out[model_out < 0] = 0
# print(model_out.cpu().detach().numpy().astype(int))
model_out = model_out.cpu().detach().numpy()[0].astype(int)

model_out = model_out[0]
# model_out = model_out[model_out>=0]
print("model_out size", model_out)

pm = piano_roll_to_pretty_midi(model_out)
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)


reconstructed


NameError: ignored

In [ ]:

torch.save(model.state_dict(), "model.pth")
from google.colab import files
files.download('model.pth')

# model Eval

In [28]:
#few helpers
def get_code(model,input):
    model(input)
    return model.code


def play_example(input):
    input = input.cpu().detach().numpy()[0].astype(int)

    # model_out = model_out[model_out>=0]
    # print("model_out size", model_out)

    pm = piano_roll_to_pretty_midi(input)
    IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)

#make it a keep top n 
def exchange_max_rows(A,B):
    maxrowA = torch.argmax(A.sum(1))
    maxrowB = torch.argmax(B.sum(1))
    rowA = A[maxrowA:maxrowA+1,].clone()
    rowB = B[maxrowB:maxrowB+1,].clone()
    # print("rowA ",rowA )
    # print("rowB ",rowB )
    A[maxrowB:maxrowB+1,] =  rowB
    B[maxrowA:maxrowA+1,] =  rowA
    return A,B

#make it a keep top n 
def keep_topk(A,k):
    mask = torch.zeros(A.shape).to(device)
    v, i  = torch.topk(A.sum(1), k)
    print("\n index is", i)
    mask[i, ] = True
    return mask * A


import torch.nn.functional as F

def get_tokcos(X, y, k, similar=True, ):
    '''
    X : Array Rows of possibilities 
    Y : vector of intrest 
    '''
    cos = nn.CosineSimilarity(eps=1e-6)
    dist = F.cosine_similarity(X, y, dim=-1)
    index_sorted = torch.argsort(dist, descending=similar)

    # we dont want identical vec
    if similar: top_k = index_sorted[1:k+1] 
    else: top_k = index_sorted[0:k] 

    return top_k

In [32]:
index_example = 747

raw_input = chord_train_dataset[index_example][None, None,:,0:5000].to(device).double()
raw_input2 = chord_train_dataset[index_example+1][None, None,:,0:5000].to(device).double()
raw_input = raw_input  / 111.14747885919755
raw_input2 = raw_input2  / 111.14747885919755

#get raw_input
# raw_input = train_data[index_example:index_example+1,:,:] 
# raw_input2 = train_data[index_example+1:index_example+2,:,:] 



#example 1
print("Orginal 1")
input1 = torch.cat([raw_input], axis=-1)
# play_example(input1)
code1 = get_code(model, input1)




#example 2
print("Orginal 2")
input2 = torch.cat([raw_input2 ], axis=-1)
# play_example(input2)
code2 = get_code(model, input2)



# New music
print("New")
print(code1.shape)
print(code2.shape)
# new_code1, new_code2  = exchange_max_rows(code1[0],code2[0])

new_code1 = (code1[0] + code2[0])


input = model.decoder(new_code1[None]) * 111.14747885919755
# input = (raw_input + raw_input2) * 111.14747885919755
# input = input.cpu().detach().numpy()[0].astype(int) 
input = input.cpu().detach().numpy()[0][0].astype(int) 


# input = input[input>=0]
# print("model_out size", model_out)

pm = piano_roll_to_pretty_midi(input)
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)




Orginal 1
Orginal 2
New
torch.Size([1, 88, 1, 412])
torch.Size([1, 88, 1, 412])


In [35]:
#@title only play most active n_atoms
print(new_code1.shape)
n_atoms = 3
onekernel = keep_topk(new_code1[0].clone(), n_atoms)
input = model.decoder(onekernel[None]) * 111.14747885919755
input = input.cpu().detach().numpy()[0].astype(int) 

pm = piano_roll_to_pretty_midi(input)
IPython.display.Audio(pm.synthesize(fs=16000), rate=16000)



torch.Size([88, 1, 412])


RuntimeError: ignored

In [ ]:
import libfmp.c1
score = libfmp.c1.midi_to_list(pm)

libfmp.c1.visualize_piano_roll(score, figsize=(8, 3))



# explore kernels


In [ ]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    #symbol_sequence= ['circle-open', 'circle', 'circle-open-dot', 'square']
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"],
                                    mode='markers',
                                    marker_size=5,
                                    marker_symbol=all_history[i]["marker_symbol"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()

In [ ]:
print(model.get_kernels().shape)


kernel1 = model.get_kernels().cpu().numpy()[1].tolist()
kernel0 = model.get_kernels().cpu().numpy()[0].tolist()
kernel6 = model.get_kernels().cpu().numpy()[6].tolist()


kernels6_plot  = {"legend": "original", 
                 "x": list(range(0,len(kernel6))), 
                 "y": kernel6,
                 "marker_symbol": 'triangle-up-open'}

#perfect plot
kernels1_plot  = {"legend": "original", 
                 "x": list(range(0,len(kernel1))), 
                 "y": kernel1,
                 "marker_symbol": 'star'}

plot([kernels6_plot], "kernels_plot")